# LAB 02.03 - Pandas

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


replicating local resources


In [69]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

logging in as daniela.pavas1@udea.edu.co... please wait
using course session ai4eng.v1.udea.r2::2023.1
success!! you are logged in


In [71]:
import numpy as np
import pandas as pd

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [4]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)
    
    if missing:
        nmissing = np.random.randint(len(price)//2)+2                                     
        price[np.random.permutation(len(price))[:nmissing]] = np.nan
    
    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]
        
    return d

In [5]:
d = create_df()
d

,price,category,margin
itemid,,,
94977,106.11,2.0,9.81
37366,88.26,2.0,11.03
76362,88.52,1.0,9.80
49051,106.38,2.0,11.12
21580,95.06,1.0,10.01
54873,113.83,2.0,10.64
87390,88.26,2.0,11.37
14773,91.23,1.0,8.59
20286,107.77,1.0,11.18


In [128]:
def select_items(df):
    df = df.copy()
    if 'margin' in df.columns:
        filtered_df = df[(df['price'] > 100) | (df['margin'] > 10)]
    else:
        filtered_df = df[df['price'] > 100]
    index_list = filtered_df.index.values
    return index_list.tolist()

**manually check your answer**

In [129]:
d = create_df()
d

,price,category
itemid,,
3789,111.02,0.0
60472,98.67,2.0
95449,101.90,1.0
4863,93.72,1.0
48281,104.84,0.0
87967,83.39,1.0
54475,84.85,2.0
99507,85.02,1.0
28307,105.21,2.0


In [130]:
select_items(d)

[3789, 95449, 48281, 28307]

**submit your code**

In [131]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category. 

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [66]:
def get_stats(df):
    df = df.copy()

    # Calcular estadísticos por categoría
    stats = df.groupby('category')['price'].agg(['mean', 'max', 'min'])

    stats= stats.rename(columns={'mean': 'media', 'max': 'maximo', 'min': 'minimo'})
    stats.index.rename('categoria',inplace=True)

    # Cambiar el tipo de índice a int

    stats.index = stats.index.astype(int)

    return stats

**manually check your answer**

In [61]:
d = create_df()
d

,price,category
itemid,,
89981,76.73,0.0
72994,94.38,1.0
91167,90.14,1.0
39084,90.92,2.0
93512,122.27,0.0
24967,103.62,1.0
21025,99.26,0.0
79225,112.15,2.0
22553,104.21,1.0


In [67]:
get_stats(d)

,media,maximo,minimo
categoria,,,
0,99.420000,122.27,76.73
1,98.087500,104.21,90.14
2,104.433333,112.15,90.92


**submit your code**

In [72]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [73]:
d = create_df(missing=True)
d

,price,category
itemid,,
97829,102.39,0.0
7285,96.30,1.0
19381,97.71,2.0
23525,117.83,1.0
93124,98.59,1.0
9698,NaN,2.0
18725,NaN,2.0
4008,105.36,1.0
15436,91.69,0.0


In [83]:
def fillna(df):
    df = df.copy()
    # compute the mean and std of the available prices
    mean_price = df['price'].mean()
    std_price = df['price'].std()

    # fill missing prices with samples from a normal distribution
    mask = df['price'].isna()
    num_missing = mask.sum()
    df.loc[mask, 'price'] = np.random.normal(mean_price, std_price, size=num_missing)
    return df

**check manually your code**

In [84]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
10393,73.59,1.0,9.32
18926,NaN,1.0,9.69
4031,NaN,1.0,10.84
97926,NaN,2.0,10.78
49320,NaN,2.0,9.63
18947,113.01,1.0,8.99
98980,92.52,1.0,8.89
20415,95.31,1.0,9.64
99098,90.08,2.0,11.13


In [85]:
fillna(d)

,price,category,margin
itemid,,,
10393,73.590000,1.0,9.32
18926,91.845023,1.0,9.69
4031,100.951341,1.0,10.84
97926,111.749469,2.0,10.78
49320,91.298822,2.0,9.63
18947,113.010000,1.0,8.99
98980,92.520000,1.0,8.89
20415,95.310000,1.0,9.64
99098,90.080000,2.0,11.13


**submit your code**

In [86]:
student.submit_task(globals(), task_id="task_03");